In [1]:
# The variable code is then sent to an api that uses base64 to decrypt the transactions
# After the decrypted data is return it is assigned to a new json file in the decrypted_files directory
# The json file is then pull from the directory to be cleaned and prepared for postgres migration
# The attributes with the json file are pulled and isolated using pandas
import json
import requests
import shutil
import os
import logging

block_number = '1194331'  # Change this number if you want to load a different file
file_path = f'C:/Users/lmhmo/Indexer_Project/indexer_project/blocks/{block_number}.json'

try:
    if os.path.exists(file_path):
        print(f"File '{file_path}' exists. Proceeding with further actions.")
        # Add your additional code here

        # Configure logging to write to both log.txt and error.txt
        logging.basicConfig(filename='log.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logging.info(f"File '{file_path}' exists. Proceeding with further actions.")

    else:
        print(f"File '{file_path}' does not exist.")
        # Log the error message to error.txt
        with open('error.txt', 'a') as error_file:
            error_file.write(f"File '{file_path}' does not exist.\n")

except Exception as e:
    print(f"An error occurred: {str(e)}")
    # Log the error message to error.txt
    with open('error.txt', 'a') as error_file:
        error_file.write(f"An error occurred: {str(e)}\n")


File 'C:/Users/lmhmo/Indexer_Project/indexer_project/blocks/1194331.json' exists. Proceeding with further actions.


In [2]:
#Isolate txs component of json file and assign it to a variable
def process_transaction_data(file_path):
    try:
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
            transaction = data['block']['data']['txs']

            if len(transaction) > 0:
                return ''.join(map(str, transaction))  # Remove unnecessary symbols
            else:
                return "No transactions found."
    except FileNotFoundError:
        return "File not found."

# Specify the file path
file_path = f'C:/Users/lmhmo/Indexer_Project/indexer_project/blocks/{block_number}.json'

try:
    code = process_transaction_data(file_path)
    print(code)  # Print the result (you can modify this)

    # Configure logging to write to both log.txt and error.txt
    logging.basicConfig(filename='log.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info(f"Data processed successfully from file '{file_path}'")

except Exception as e:
    print(f"An error occurred: {str(e)}")
    # Log the error message to error.txt
    with open('error.txt', 'a') as error_file:
        error_file.write(f"Error occurred: {str(e)}\n")


CqEHCt4DCh4vY29zbW9zLmF1dGh6LnYxYmV0YTEuTXNnR3JhbnQSuwMKLm1pZ2Fsb28xcGVtdDYzemN0cHh0bWFyZXZyamgwbXdydjltYXBucHkzNmtxd2ESLm1pZ2Fsb28xZjQ5eHEwcm1haDM5c2s1OGFheHE2Z25xY3Z1cGVlN2puZTB3ZXAa2AIKxwIKKi9jb3Ntb3Muc3Rha2luZy52MWJldGExLlN0YWtlQXV0aG9yaXphdGlvbhKYAhKTAgo1bWlnYWxvb3ZhbG9wZXIxOWF2OTNmbGg1MzhlcmpuM3U4Z2ptaHVmYWFhODJ4NmFyNDJyeDUKNW1pZ2Fsb292YWxvcGVyMXhlc3FyOHZqdnkzNGpodTAyN3pkNzB5cGwwbm5ldjVlZWM5MGV3CjVtaWdhbG9vdmFsb3BlcjFzZ3V2YTRjZjludGZ2ZWx3Y2tzMDRwY3Z4ano4dnk2eWRnY2Nwego1bWlnYWxvb3ZhbG9wZXIxNXkzdDU1ZWpnZW41Y2ZtczQ5c251OWp2a3ZlNDVmbWE1YWg5OTgKNW1pZ2Fsb292YWxvcGVyMTl2aHVuOXJuNTdmOTRtbnJ5eWpyNW5heGhrdDg3cmh4MmQ5NGMyIAESDAi4xbKmBhCAs+3QAQr8AQoeL2Nvc21vcy5hdXRoei52MWJldGExLk1zZ0dyYW50EtkBCi5taWdhbG9vMXBlbXQ2M3pjdHB4dG1hcmV2cmpoMG13cnY5bWFwbnB5MzZrcXdhEi5taWdhbG9vMWY0OXhxMHJtYWgzOXNrNThhYXhxNmducWN2dXBlZTdqbmUwd2VwGncKZwoqL2Nvc21vcy5hdXRoei52MWJldGExLkdlbmVyaWNBdXRob3JpemF0aW9uEjkKNy9jb3Ntb3MuZGlzdHJpYnV0aW9uLnYxYmV0YTEuTXNnV2l0aGRyYXdEZWxlZ2F0b3JSZXdhcmQSDAi4xbKmBhCAs+3QAQqiAQojL2Nvc21v

In [3]:
# This section hashes the txs variable and also translates the variable with an api
import hashlib
import base64
import requests
import json
import shutil

def hash_to_hex(data: str) -> str:
    try:
        # Convert data from base64 to bytes
        data_bytes = base64.b64decode(data)
        # Calculate SHA-256 hash
        sha256_hash = hashlib.sha256(data_bytes).hexdigest()
        # Convert hash to uppercase
        return sha256_hash.upper()
    except Exception as e:
        print(f"Error while hashing: {e}")
        return None

def decode_transactions(tx: str, block: int, i: int) -> str:
    try:
        url = "https://phoenix-lcd.terra.dev/cosmos/tx/v1beta1/decode"
        headers = {'Content-Type': 'application/json'}
        data = json.dumps({"tx_bytes": tx})

        response = requests.post(url, headers=headers, data=data)
        decoded_response = response.json()

        # Add hashed_result to the decoded data
        decoded_response["hashed_result"] = hashed_result

        # Save the updated decoded data to a JSON file
        filename = f"{block}.json"
        with open(filename, "w") as json_file:
            json.dump(decoded_response, json_file, indent=2)

        print(f"Transaction {i + 1} decoded and saved to {filename}")
        return filename
    except Exception as e:
        print(f"Error while decoding: {e}")
        return None

# Example usage
code = code # Replace with actual base64-encoded data
block_number = block_number # Replace with the block number

# Hash the data
hashed_result = hash_to_hex(code)
print(f"Hashed result: {hashed_result}")

# Decode transactions
code_list = code.split()
many = len(code_list)

print(f"Number of transactions: {many}")

# Store generated filenames in a list
generated_filenames = []
for i, tx in enumerate(code_list):
    filename = decode_transactions(tx, block_number, i)
    if filename:
        generated_filenames.append(filename)

# Move the generated JSON files to the "Decrypted_files" directory
target_directory = "C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files"
for filename in generated_filenames:
    shutil.move(filename, target_directory)
    print(f"File '{filename}' moved to '{target_directory}'.")



Hashed result: 06EB3B55A212899E985408A34F185D1076DA90EF9D0EEB9C1BD1201AA9023DC8
Number of transactions: 1
Transaction 1 decoded and saved to 1194331.json
File '1194331.json' moved to 'C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files'.
